In [118]:
import numpy as np
import torch
class BloatWareConvTranspose2D():
    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, padding=0, output_padding=0, groups=1, bias=True, dilation=1, dtype=None):
        
        if isinstance(in_channels, int) and in_channels > 0:
            self.in_channels = in_channels
        else:
            raise ValueError("Invalid in_channels")
        
        if isinstance(out_channels, int) and out_channels > 0:
            self.out_channels = out_channels
        else:
            raise ValueError("Invalid out_channels")
        
        if isinstance(groups, int) and groups > 0:
            self.groups = groups
        else:
            raise ValueError("Invalid groups")

        if isinstance(stride, int) and stride > 0:
            self.stride = stride
        else:
            raise ValueError("Invalid stride")
        
        if isinstance(padding, int) and padding > -1:
            self.padding = padding
        else:
            raise ValueError("Invalid padding")
            
        if isinstance(output_padding, int) and output_padding > -1:
            self.output_padding = output_padding
        else:
            raise ValueError("Invalid output_padding")
            
        if isinstance(dilation, int) and dilation > 0:
            self.dilation = dilation
        else:
            raise ValueError("Invalid dilation")
            
        if not((self.in_channels % self.groups == 0) and (self.out_channels % self.groups == 0)):
            raise ValueError("in_channels and out_channels must both be divisible by groups")
        
        if (self.output_padding >= self.dilation and self.output_padding >= self.stride) or (self.output_padding >= self.dilation and self.output_padding >= self.stride):
            raise ValueError("output_padding should be smaller than dilation or stride")

        if bias == True:
          self.bias = torch.rand(self.out_channels)
        else:
          self.bias = torch.zeros(self.out_channels)
    
        if isinstance(kernel_size, int):
            self.weight = torch.rand(
                self.in_channels,
                self.out_channels,
                kernel_size,
                kernel_size,
            )
            self.kernel_size = kernel_size
        else:
            raise ValueError("kernel size must be int or tuple")

        self.dtype = dtype
    
    def forward(self, input_tensor):
        result = []

        for l in range(self.out_channels):
    
          feature_map = torch.zeros((input_tensor.shape[1]-1)*self.stride + self.dilation * (self.kernel_size-1)+1, (input_tensor.shape[2]-1)*self.stride  + self.dilation * (self.kernel_size-1)+1 ) #генерация пустой feature-map
          for c in range (self.in_channels):
    
            for i in range (0, input_tensor.shape[1]):  #проход по всем пикселям изображения
              for j in range (0, input_tensor.shape[2]):
    
                val = input_tensor[c][i][j]
                proizv = val*self.weight[c][l]
    
                zero_tensor = torch.zeros((self.weight.shape[2]-1)*self.dilation+1, (self.weight.shape[3]-1)*self.dilation+1)
    
                for a in range (0, zero_tensor.shape[0], self.dilation):
                  for b in range (0, zero_tensor.shape[1], self.dilation):
                    zero_tensor[a][b] = proizv[a//self.dilation][b//self.dilation]
    
                res = np.add((zero_tensor), feature_map[i*self.stride:i*self.stride+(self.weight.shape[2]-1)*self.dilation+1, j*self.stride:j*self.stride+(self.weight.shape[3]-1)*self.dilation+1])
                feature_map[i*self.stride:i*self.stride+(self.weight.shape[2]-1)*self.dilation+1, j*self.stride:j*self.stride+(self.weight.shape[3]-1)*self.dilation+1] = res
    
    
          result.append(np.add(feature_map, np.full((feature_map.shape), self.bias[l])))
    
    
        for t in range(len(result)):
          if self.output_padding > 0:
            pad_func = torch.nn.ConstantPad1d((0, self.output_padding, 0, self.output_padding), 0)
            result[t] = pad_func(result[t])
    
          result[t] = result[t][0+self.padding:result[t].shape[0]-self.padding, 0+self.padding:result[t].shape[1]-self.padding]

        return np.array(result)

In [119]:
torchConvTranspose2D = torch.nn.ConvTranspose2d(15, 5, 3, stride=1, padding=1, dilation=1)
input_image = torch.randn(15, 50, 50)
output = torchConvTranspose2D(input_image)

In [120]:
myConvTranspose2D = BloatWareConvTranspose2D(15, 5, 3, stride=1, padding=1, dilation=1)

In [121]:
myConvTranspose2D.weight = torchConvTranspose2D.weight.detach().numpy()
myConvTranspose2D.bias = torchConvTranspose2D.bias.detach().numpy()

In [122]:
output_mock = myConvTranspose2D.forward(input_image)

In [127]:
output

tensor([[[ 9.4144e-02, -4.0830e-01,  1.3812e-01,  ...,  3.5826e-01,
          -3.6301e-01, -6.4993e-02],
         [-5.9811e-01,  1.0341e+00,  7.8236e-01,  ..., -2.6313e-01,
          -1.2244e-01,  6.8067e-01],
         [ 1.2285e+00, -1.9864e+00, -7.7189e-01,  ..., -9.3731e-01,
          -4.7767e-01,  4.5403e-01],
         ...,
         [-1.4497e-01, -6.2573e-01, -3.1614e-01,  ..., -5.7800e-01,
          -1.1800e+00,  4.7009e-01],
         [-1.8099e+00, -1.4266e+00, -1.3338e+00,  ..., -2.2716e-01,
           3.2290e-01,  6.4719e-02],
         [-5.1554e-01,  5.6257e-01, -1.2997e+00,  ...,  8.3713e-01,
          -2.6759e-01, -6.0303e-01]],

        [[ 1.9259e-01,  6.9018e-02, -3.2970e-01,  ...,  2.5642e-01,
           7.8945e-01,  1.4418e-01],
         [ 3.6651e-01, -1.2020e-01,  1.2002e+00,  ..., -1.4035e+00,
          -7.4276e-02,  6.1441e-01],
         [-8.2292e-01, -2.6048e-01,  7.5872e-01,  ...,  8.0376e-01,
          -5.7097e-02, -2.7008e-01],
         ...,
         [ 3.0044e-01,  5

In [130]:
result_test = output.detach().numpy().astype("float16") == output_mock.astype(
    "float16"
)

In [131]:
result_test

array([[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],

       [[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  Tr